In [101]:
import pandas as pd , numpy as np , quandl

# 1-step = setupping and cleaning the data

#stock = pd.read_csv('ADANIPORTS.csv' , index_col = 'Date' ,parse_dates = True)
gdp = quandl.get('FRED/GDP'  ,returns = 'numpy' , rows = 5)
gdp

gdp = quandl.get('FRED/GDP'  ,rows = 5)

#to change sampling freq:
gdp = quandl.get('FRED/GDP'  ,collapse = 'monthly')
gdp

#for elementry calculation
gdp = quandl.get('FRED/GDP'  , transformation = 'rdiff')
gdp

stock = quandl.get('NSE/OIL.1' )
stock

stock = quandl.get('EIA/PET_RWTC_D' , start_date = '2014-12-31' , end_date = '2021-12-31')
stock.head()

stock = quandl.get(['EIA/PET_RWTC_D' , 'NSE/OIL.1' ] ,start_date = '2014-12-31' , end_date = '2021-12-31')
stock

mydata = quandl.get_table('ZACKS/FC' , ticker = 'AAPL')
mydata = quandl.get_table('ZACKS/FC' , peginate = True)
mydata


stock = quandl.get('WiKi/FB' , start_date = '2014-12-31' , end_date = '2021-12-31')
stock.to_csv('wiki FB stock.csv')

#stock = pd.read_csv('wiki FB stock.csv')
stock = quandl.get('WiKi/AMZN' ,start_data = '2015-01-01' , end_date = '2021-12-31')
stock.to_csv('data/WiKi Amozon.csv')

In [138]:
#stock = pd.read_csv('data\WiKi Amozon.csv')
stock = pd.read_csv('data\wiki FB stock.csv' )
stock.head(3)

,Date,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
0,2014-12-31,79.54,79.8000,77.86,78.02,20040439.0,0.0,1.0,79.54,79.8000,77.86,78.02,20040439.0
1,2015-01-02,78.58,78.9300,77.70,78.45,18177475.0,0.0,1.0,78.58,78.9300,77.70,78.45,18177475.0
2,2015-01-05,77.98,79.2455,76.86,77.19,26452191.0,0.0,1.0,77.98,79.2455,76.86,77.19,26452191.0


In [139]:
try:
    stock.index = stock['Date']
    stock.drop('Date',axis = 1,inplace=True)
except:
    pass

'From : ' + stock.index[0] + ' --> ' + stock.index[len(stock)-1]
#renaming column 
#stock.rename(columns={'Volume' : 'Vol'})

'From : 2014-12-31 --> 2018-03-27'

In [104]:
# get the adjusted close price
open = stock[['Open','Adj. Open']] 
open.head(3)

,Open,Adj. Open
0,22.38,1.865000
1,20.50,1.708333
2,20.75,1.729167


In [105]:
close = stock[['Adj. Close']]
close.head(3)

,Adj. Close
0,1.729167
1,1.708333
2,1.635833


stock.info()

In [106]:
# a varialble for predicting 'n' days out into the future
forecast_out = 30

# another coumn (the target) shifted 'n' units up
close['Prediction'] = close.shift(-forecast_out) 
close.dropna(inplace=True)
close.head() , close.tail()

(   Adj. Close  Prediction
 0    1.729167    1.541667
 1    1.708333    1.515833
 2    1.635833    1.588333
 3    1.427500    1.911667
 4    1.395833    2.000000,
       Adj. Close  Prediction
 5213     1442.84     1581.86
 5214     1416.78     1544.10
 5215     1350.47     1495.56
 5216     1338.99     1555.86
 5217     1386.23     1497.05)

# 2-step =  spliting the data

In [107]:
X_set = np.array(close[['Adj. Close']])

Y_set = np.array(close['Prediction'])

In [108]:
#spliting in 4:1 (80:20) the data set
from sklearn.model_selection import train_test_split

X_train , X_test , Y_train , Y_test = train_test_split(X_set , Y_set , test_size = 0.2) # test_size = 20%
len(X_train) ,'+', len(X_test) ,'=', len(X_set)

(4174, '+', 1044, '=', 5218)

# 3-step =  training the data by :
##  1-svm (SVR) (support vector machine)
##  2-linear regression
##  

In [109]:
#1 - #Support Vector machine (Regressor)
from sklearn.svm import SVR 

svr_rbf = SVR(kernel= 'rbf' , C=1e3 , gamma=0.1 ) 
svr_rbf.fit(X_train , Y_train) #SVR(C=1000.0, gamma=0.1)
print('Score of SVR model : ',svr_rbf.score(X_test , Y_test) * 100 ,'%')

Score of SVR model :  94.61822640869782 %


In [110]:
#2 - linear regression
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train , Y_train)
lr.score(X_test , Y_test)

print('Score of lr model : ',lr.score(X_test , Y_test) * 100 ,'%')

Score of lr model :  98.78166798628982 %


In [111]:
#set x_forecast equl to the last 30 rows of the original data set from Adf. Close column
x_forecast = np.array(close[['Adj. Close']])[-forecast_out:]
print(type(x_forecast) , x_forecast.shape)
x_forecast[:4]

#without reshape : shape = (30,1)
#with .reshape(1,-1) : shape = (1, 30)
#with .reshape(2,-2) : shape = (2, 15)
#with .reshape(3,-2) : shape = (3,10)

<class 'numpy.ndarray'> (30, 1)


array([[1169.47],
       [1189.01],
       [1204.2 ],
       [1209.59]])

In [112]:
# predi for the last n days by linear SVR 
svr_pred = svr_rbf.predict(x_forecast)
svr_pred

array([1414.41042621, 1450.94977818,  711.77738827, 1448.5902625 ,
       1468.2502081 , 1482.82019319, 1484.65989942, 1499.89972274,
       1521.85014717,  668.73668296,  668.74509646, 1493.5500673 ,
       1403.01446389, 1513.7927143 , 1537.53976974, 1544.89977197,
       1551.75987368, 1578.79022431, 1598.28990369, 1475.89842847,
        750.52321883, 1582.22012176,  879.31597418, 1544.82967185,
       1586.41000322, 1581.76000096, 1544.00003503, 1495.45978048,
       1555.75986738, 1496.95008759])

In [113]:
# predi for the last n days by linear regression mode
lr_pred = lr.predict(x_forecast)
lr_pred

array([1239.61297102, 1260.38052901, 1276.52480771, 1282.25342274,
       1303.03160895, 1321.87545763, 1328.07171469, 1329.8041159 ,
       1353.55820602, 1383.86991297, 1383.50855321, 1373.02912016,
       1371.24357782, 1372.58273458, 1407.36892562, 1444.81217372,
       1439.46617491, 1461.19027344, 1486.80430351, 1503.41622425,
       1524.82147593, 1539.52031559, 1473.99728847, 1517.57302427,
       1481.03317557, 1530.15684651, 1502.45968371, 1431.98390223,
       1419.7826962 , 1469.99044643])

# saving the model

In [114]:
import joblib

joblib.dump(svr_rbf , 'SVR Amzn stock.joblib')
joblib.dump(lr , 'LR Amzn stock.joblib')

In [ ]:
joblib.dump(svr_rbf , 'SVR FB stock.joblib')
joblib.dump(lr , 'LR FB stock.joblib')